In [ ]:
import re
data = []

# request page may pha ca phe
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('-headless') 
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("--incognito")
i = 1
while True:
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.implicitly_wait(30)

  tiki_url = f'https://tiki.vn/may-pha-ca-phe/c1939?page={i}&src=c.1882.hamburger_menu_fly_out_banner'
  print('this is -----------------------------------------------',tiki_url)
  driver.get(tiki_url)

  html_data = driver.page_source

  driver.close()
    
  # parse html into beautiful soup
  from bs4 import BeautifulSoup

  # r.text is a HTML file so use html.parser
  soup = BeautifulSoup(html_data, 'html.parser')
  # total number of items:  
  result_items = soup.find('div', {'class':"CategoryViewstyle__Right-bhstkd-1 jJGphj"}).div.div.div.h4.text.split()[0]
  # scripts of all item in page 1     
  scripts = soup.find_all('script', {'type': 'application/ld+json'})

  if int(len(data)) >= int(result_items):
    break
  else:
    for a in range(1,len(scripts)-1): 
      test = str(scripts[a])

      # find product_id
      pattern = r'.*"sku":"(\d+)".*'
      product_id1 = str(re.findall(pattern, test)).strip('[')
      product_id2 = product_id1.strip(']')
      product_id = product_id2.strip("'")

      # file title
      pattern = r'"name":"(.*)","des'
      title1 = str(re.findall(pattern, test)).strip('[')
      title2 = title1.strip(']')
      title = title2.strip("'")

      # find price
      pattern = r'"price":(\d+)}'
      price1 = str(re.findall(pattern, test)).strip('[')
      price2 = price1.strip(']')
      price = price2.strip("'")

      # find image
      pattern = r'"image":"(.*g)","name'
      image_url1 = str(re.findall(pattern, test)).strip('[')
      image_url2 = image_url1.strip(']')
      image_url = image_url2.strip("'")

      # find product_url
      pattern = r'"url":"(.*)","image"'
      product_url1 = str(re.findall(pattern, test)).strip('[')
      product_url2 = product_url1.strip(']')
      product_url = product_url2.strip("'")

      # create data contain dict of 48 items
      d_script = {'product_id':'', 'title':'', 'price':'', 'image_url':'', 'product_url':''}
      try:
          d_script['product_id'] = product_id
          d_script['title'] = title
          d_script['price'] = price
          d_script['image_url'] = image_url
          d_script['product_url'] = product_url
          data.append(d_script)

      except:
          print("We got one article error!")
  i += 1


In [ ]:
len(data)

In [ ]:
# create dataframe
import pandas as pd
 
product = pd.DataFrame(data = data)  

In [ ]:
product